<a href="https://colab.research.google.com/github/aruaru0/Foursquare-Location-Matching/blob/main/IOU%E6%A4%9C%E8%A8%8E.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# for Colab

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!mkdir -p ~/.kaggle
!cp /content/drive/MyDrive/datas/kaggle.json  ~/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json

In [ ]:
!pip install kaggle
!apt install unzip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Reading package lists... Done
Building dependency tree       
Reading state information... Done
unzip is already the newest version (6.0-21ubuntu1.1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [ ]:
!mkdir input output

In [ ]:
!kaggle competitions download -c foursquare-location-matching
!unzip foursquare-location-matching.zip -d input/foursquare-location-matching

 86% 137M/159M [00:04<00:00, 28.9MB/s]
100% 159M/159M [00:05<00:00, 32.7MB/s]
Archive:  foursquare-location-matching.zip
  inflating: input/foursquare-location-matching/pairs.csv  
  inflating: input/foursquare-location-matching/sample_submission.csv  
  inflating: input/foursquare-location-matching/test.csv  
  inflating: input/foursquare-location-matching/train.csv  


In [ ]:
!pip install Levenshtein
!pip install pykakasi
!pip install fuzzywuzzy
!pip install cdifflib polyleven

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 258 kB 4.2 MB/s 
     |████████████████████████████████| 2.0 MB 40.5 MB/s 
     |████████████████████████████████| 102 kB 61.9 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.4 MB 4.4 MB/s 
  Created wheel for jaconv: filename=jaconv-0.3-py3-none-any.whl size=15564 sha256=7c283ba08d2a924a556279d897a6abfe21ee3df17d8e92541f12b2f55f709f2f
  Stored in directory: /root/.cache/pip/wheels/8f/4f/c2/a2a3b14d0e94f855f4aa8887bf0267bee9ecfb8e62a9ee2d92
Successfully built jaconv
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for cdifflib: filename=cdifflib-1.2.5-cp37-cp37m-linux_x86_64.whl size=32982 

# change directory

In [44]:
%cd /content/output

/content/output


# start code

In [45]:
## Imports
import warnings
warnings.filterwarnings('ignore')

import os
import gc
import time
import random
import Levenshtein
import difflib
import multiprocessing
import pandas as pd
import numpy as np
import lightgbm as lgb
from tqdm.auto import tqdm
from sklearn.model_selection import GroupKFold
from sklearn.neighbors import KNeighborsRegressor
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
import pykakasi
from tqdm.notebook import tqdm
import math

from fuzzywuzzy import fuzz

from collections import Counter

from polyleven import levenshtein
from cdifflib import CSequenceMatcher

tqdm.pandas()

## Conversion kanji to alphabet

In [46]:
def convert_japanese_alphabet(df: pd.DataFrame):
    kakasi = pykakasi.kakasi()
    kakasi.setMode('H', 'a')  # Convert Hiragana into alphabet
    kakasi.setMode('K', 'a')  # Convert Katakana into alphabet
    kakasi.setMode('J', 'a')  # Convert Kanji into alphabet
    conversion = kakasi.getConverter()

    def convert(row):
        for column in ["name", "address", "city", "state"]:
            try:
                row[column] = conversion.do(row[column])
            except:
                pass
        return row

    df[df["country"] == "JP"] = df[df["country"] == "JP"].progress_apply(convert, axis=1)
    return df

In [47]:
def categorical_similarity(A, B):
    if not A or not B:
        return -1

    A = set(str(A).split(", "))
    B = set(str(B).split(", "))

    # Find intersection of two sets
    nominator = A.intersection(B)

    similarity_1 = len(nominator) / len(A)
    similarity_2 = len(nominator) / len(B)

    return max(similarity_1, similarity_2)

## start 

In [48]:
## Parameters
is_debug = False
SEED = 2022
num_neighbors = 10
num_neighbors2 = 6
num_neighbors3 = 30
# num_neighbors = 10
# num_neighbors2 = 5
# num_neighbors3 = 22
num_split = 10
# feat_columns = ['name', 'address', 'city', 
#             'state', 'zip', 'url', 
#            'phone', 'categories', 'country']
# feat_columns = ['dist', 'name', 'address', 'city', 
#             'state', 'zip', 'url', 
#            'phone', 'categories', 'country', 'categories2']
# vec_columns = ['name', 'categories', 'address', 
#                'state', 'url', 'country', 'categories2']
feat_columns = ['dist', 'name', 'address', 'city', 
            'phone', 'categories', 'categories2']
vec_columns = ['name', 'categories', 'address', 'categories2']
rec_columns = ['name', 'address', 'categories', 'address', 'phone']


def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    
seed_everything(SEED)

In [49]:
%load_ext Cython

The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


In [50]:
%%cython
def LCS(str S, str T):
    cdef int i, j
    cdef list dp = [[0] * (len(T) + 1) for _ in range(len(S) + 1)]
    for i in range(len(S)):
        for j in range(len(T)):
            dp[i + 1][j + 1] = max(dp[i][j] + (S[i] == T[j]), dp[i + 1][j], dp[i][j + 1], dp[i + 1][j + 1])
    return dp[len(S)][len(T)]

In [51]:
def post_process(df):
    id2match = dict(zip(df['id'].values, df['matches'].str.split()))

    for base, match in df[['id', 'matches']].values:
        match = match.split()
        if len(match) == 1:        
            continue

        for m in match:
            if base not in id2match[m]:
                id2match[m].append(base)
    df['matches'] = df['id'].map(id2match).map(' '.join)
    return df 

# get manhattan distance
def manhattan(lat1, long1, lat2, long2):
    return np.abs(lat2 - lat1) + np.abs(long2 - long1)

# get haversine distance
def vectorized_haversine(lats1, lats2, longs1, longs2):
    radius = 6371
    dlat=np.radians(lats2 - lats1)
    dlon=np.radians(longs2 - longs1)
    a = np.sin(dlat/2) * np.sin(dlat/2) + np.cos(np.radians(lats1)) \
        * np.cos(np.radians(lats2)) * np.sin(dlon/2) * np.sin(dlon/2)
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
    d = radius * c
    return d

In [52]:
def recall_simple(df):
    threshold = 2
    
    val2id_d = {}
    for col in rec_columns:
        temp_df = df[['id', col]]
        temp_df[col] = temp_df[col].str.lower()
        val2id = temp_df.groupby(col)['id'].apply(set).to_dict()
        val2id_d[col] = val2id
        del val2id
    
    cus_ids = []
    match_ids = []
    for vals in tqdm(df[rec_columns + ['id']].fillna('null').values):
        cus_id = vals[-1]
        match_id = []
        
        rec_match_count = []
        for i in range(len(rec_columns)):
            col = rec_columns[i]
            
            if vals[i] != 'null':
                rec_match_count += list(val2id_d[col][vals[i].lower()])
        rec_match_count = dict(Counter(rec_match_count))
        
        for k, v in rec_match_count.items():
            if v > threshold:
                match_id.append(k)
        
        cus_ids += [cus_id] * len(match_id)
        match_ids += match_id
    
    train_df = pd.DataFrame()
    train_df['id'] = cus_ids
    train_df['match_id'] = match_ids
    train_df = train_df.drop_duplicates()
    del cus_ids, match_ids
    
    num_data = len(train_df)
    num_data_per_id = num_data / train_df['id'].nunique()
    print('Num of data: %s' % num_data)
    print('Num of data per id: %s' % num_data_per_id)
    
    return train_df

In [53]:
# data.groupby('categories')['categories'].count().sort_values(ascending=False)[:500]

In [54]:
def cat_index(x) : 
  if x in seld : return seld[x]
  return np.nan

In [55]:
def recall_knn(df, Neighbors = 10, Neighbors2 = 10, Neighbors3 = 10 ):
    print('Start knn grouped by country')
    train_df_country = []
    for country, country_df in tqdm(df.groupby('country')):
        country_df = country_df.reset_index(drop = True)

        neighbors = min(len(country_df), Neighbors)
        knn = KNeighborsRegressor(n_neighbors = neighbors,
                                    metric = 'haversine',
                                    n_jobs = -1)
        knn.fit(country_df[['latitude','longitude']].apply(lambda dr : (dr*math.pi)/180),
                country_df.index)
        dists, nears = knn.kneighbors(
            country_df[['latitude', 'longitude']].apply(lambda dr : (dr*math.pi)/180), 
            return_distance = True)
        # knn.fit(country_df[['latitude','longitude']],
        #         country_df.index)
        # dists, nears = knn.kneighbors(
        #     country_df[['latitude', 'longitude']], 
        #     return_distance = True)

        for k in range(neighbors):            
            cur_df = country_df[['id']]
            cur_df['match_id'] = country_df['id'].values[nears[:, k]]
            # cur_df['kdist_country'] = dists[:, k]
            cur_df['kneighbors_country'] = k
            
            train_df_country.append(cur_df)
    train_df_country = pd.concat(train_df_country)
    
    #-------------------------------------------------
    print('Start knn grouped by categories')
    sel = list(set(data.groupby('categories')['categories'].count().sort_values(ascending=False)[:500].keys()))
    sel.sort()
    seld = {x:y for y, x in enumerate(sel)}


    train_df_categories = []
  
    for categories, categories_df in tqdm(df.groupby('categories')):
        if categories not in sel : continue
        categories_df = categories_df.reset_index(drop = True)

        neighbors = min(len(categories_df), Neighbors2)
        knn = KNeighborsRegressor(n_neighbors = neighbors,
                                    metric = 'haversine',
                                    n_jobs = -1)
        knn.fit(categories_df[['latitude','longitude']].apply(lambda dr : (dr*math.pi)/180),
                categories_df.index)
        dists, nears = knn.kneighbors(
            categories_df[['latitude', 'longitude']].apply(lambda dr : (dr*math.pi)/180), 
            return_distance = True)
        # knn.fit(categories_df[['latitude','longitude']],
        #         categories_df.index)
        # dists, nears = knn.kneighbors(
        #     categories_df[['latitude', 'longitude']], 
        #     return_distance = True)

        for k in range(neighbors):            
            cur_df = categories_df[['id']]
            cur_df['match_id'] = categories_df['id'].values[nears[:, k]]
            # cur_df['kdist_categories'] = dists[:, k]
            cur_df['kneighbors_categories'] = k
            cur_df['categories_index'] = seld[categories]
            
            train_df_categories.append(cur_df)
    train_df_categories = pd.concat(train_df_categories)
    #-------------------------------------------------
    print('Start knn')
    train_df = []
    knn = NearestNeighbors(n_neighbors = Neighbors3)
    knn.fit(df[['latitude','longitude']].apply(lambda dr : (dr*math.pi)/180), df.index)
    dists, nears = knn.kneighbors(df[['latitude','longitude']].apply(lambda dr : (dr*math.pi)/180))
    # knn.fit(df[['latitude','longitude']], df.index)
    # dists, nears = knn.kneighbors(df[['latitude','longitude']])
    
    for k in range(Neighbors3):            
        cur_df = df[['id']]
        cur_df['match_id'] = df['id'].values[nears[:, k]]
        # cur_df['kdist'] = dists[:, k]
        cur_df['kneighbors'] = k
        train_df.append(cur_df)
    
    train_df = pd.concat(train_df)
    train_df = train_df.merge(train_df_country,
                                 on = ['id', 'match_id'],
                                 how = 'outer')
    #-------------------------------------------------
    train_df = train_df.merge(train_df_categories,
                                on = ['id', 'match_id'],
                                how = 'outer')

    del train_df_country
    
    return train_df, seld

### add features

In [56]:
def add_features(df):    
    for col in tqdm(feat_columns):
        if col == 'dist':
            lat = data.loc[df['id']]['latitude'].values
            match_lat = data.loc[df['match_id']]['latitude'].values
            lon = data.loc[df['id']]['longitude'].values
            match_lon = data.loc[df['match_id']]['longitude'].values
            df['latdiff'] = (lat - match_lat)
            df['londiff'] = (lon - match_lon)
            df['manhattan'] = manhattan(lat, lon, match_lat, match_lon)
            df['euclidean'] = (df['latdiff'] ** 2 + df['londiff'] ** 2) ** 0.5
            df['haversine'] = vectorized_haversine(lat, match_lat, lon, match_lon)
            continue
        
        col_values = data.loc[df['id']][col].values.astype(str)
        matcol_values = data.loc[df['match_id']][col].values.astype(str)
        
        if col in vec_columns:
            tv_fit = tfidf_d[col]
            indexs = [id2index_d[i] for i in df['id']]
            match_indexs = [id2index_d[i] for i in df['match_id']]                    
            df[f'{col}_sim'] = tv_fit[indexs].multiply(tv_fit[match_indexs]).\
                                            sum(axis = 1).A.ravel()
        
        geshs = []
        levens = []
        jaros = []
        lcss = []
        fuzwuz = []
        for s, match_s in zip(col_values, matcol_values):
            if s != 'nan' and match_s != 'nan':                   
                # geshs.append(difflib.SequenceMatcher(None, s, match_s).ratio())
                # levens.append(Levenshtein.distance(s, match_s))
                geshs.append(CSequenceMatcher(None, s, match_s).ratio())
                levens.append(levenshtein(s, match_s))
                jaros.append(Levenshtein.jaro_winkler(s, match_s))
                lcss.append(LCS(str(s), str(match_s)))
                fuzwuz.append(fuzz.ratio(s, match_s))
            else:
                geshs.append(np.nan)
                levens.append(np.nan)
                jaros.append(np.nan)
                lcss.append(np.nan)
                fuzwuz.append(np.nan)
        
        df[f'{col}_gesh'] = geshs
        df[f'{col}_leven'] = levens
        df[f'{col}_jaro'] = jaros
        df[f'{col}_lcs'] = lcss
        df[f'{col}_fuz'] = fuzwuz
        
        # add 2022.06.26
        # print(col)
        if col == 'categories' : 
          cats = []
          for s, match_s in zip(col_values, matcol_values):
            cats.append(categorical_similarity(s, match_s))
            # print(s,":", match_s, categorical_similarity(s, match_s))
          df[f'{col}'] = cats


        if col not in ['phone', 'zip']:
            df[f'{col}_len'] = list(map(len, col_values))
            df[f'match_{col}_len'] = list(map(len, matcol_values)) 
            df[f'{col}_len_diff'] = np.abs(df[f'{col}_len'] - df[f'match_{col}_len']) /\
                                        df[f'{col}_len'] 
            df[f'{col}_nleven'] = df[f'{col}_leven'] / \
                                    df[[f'{col}_len', f'match_{col}_len']].max(axis = 1)
            
            df[f'{col}_nlcsk'] = df[f'{col}_lcs'] / df[f'match_{col}_len']
            df[f'{col}_nlcs'] = df[f'{col}_lcs'] / df[f'{col}_len']
            
            df = df.drop(f'{col}_len', axis = 1)
            df = df.drop(f'match_{col}_len', axis = 1)
            gc.collect()
            
    return df

In [57]:
def get_id2poi(input_df: pd.DataFrame) -> dict:
    return dict(zip(input_df['id'], input_df['point_of_interest']))

def get_poi2ids(input_df: pd.DataFrame) -> dict:
    return input_df.groupby('point_of_interest')['id'].apply(set).to_dict()

def get_score(input_df: pd.DataFrame):
    scores = []
    for id_str, matches in zip(input_df['id'].to_numpy(), input_df['matches'].to_numpy()):
        targets = poi2ids[id2poi[id_str]]
        preds = set(matches.split())
        score = len((targets & preds)) / len((targets | preds))
        scores.append(score)
    scores = np.array(scores)
    return scores.mean()

def analysis(df):
    print('Num of data: %s' % len(df))
    print('Num of unique id: %s' % df['id'].nunique())
    print('Num of unique poi: %s' % df['point_of_interest'].nunique())
    
    poi_grouped = df.groupby('point_of_interest')['id'].count().reset_index()
    print('Mean num of unique poi: %s' % poi_grouped['id'].mean())

In [58]:
## Data load
data_root = '../input/foursquare-location-matching'
data = pd.read_csv(os.path.join(data_root, 'train.csv'))

if is_debug:
    data = data.sample(n = 10000, random_state = SEED)
    data = data.reset_index(drop = True)

## sort categories 0626

In [59]:
cat = []
for e in data['categories'] :
  if type(e) == float :
    cat.append(e)
    continue
  e = e.split(", ")
  e.sort()
  cat.append(", ".join(e))
data['categories'] = cat
del cat

## add categories2

In [60]:
cat_count = dict(data.groupby('categories')['categories'].count().sort_values(ascending=False))
pat = data.groupby('point_of_interest')['categories'].apply(list).apply(set)

from collections import defaultdict
import math 

pairs = defaultdict(int)
for e in pat : 
  if len(e) == 1 : continue
  x = list(e)
  for i in range(len(x)) :
    for j in range(i+1, len(x)) :
      if type(x[i]) == float and math.isnan(x[i]) : continue
      if type(x[j]) == float and math.isnan(x[j]) : continue
      pairs[(x[i], x[j])] += 1

pairs = [(pairs[x], x[0], x[1]) for x in pairs]
pairs.sort(reverse=True)

cdict = defaultdict(str)
cnt = 0
for n, x, y in pairs :
  if n < 100: break
  nx, ny = cat_count[x], cat_count[y]
  # print(n, x,"-", y, nx, ny)
  if nx < ny : 
    nx, ny = ny, nx
    x,y = y,x
  if y not in cdict :
    if x in cdict :
      x = cdict[x]
    cdict[y] = x
  # print(n, x,"-", y, nx, ny)
  # if cnt == 10: break
  cnt+=1

def f(x) :
  if type(x)==float and math.isnan(x) : return x
  if x in cdict : return cdict[x]
  return x

data['categories2'] = data['categories'].apply(f)

In [61]:
# 変換に使った辞書を表示（使うので）
cdict

defaultdict(str,
            {'ATMs': 'Banks',
             'ATMs, Banks': 'Banks',
             'Airport Services': 'Airports',
             'Airport Terminals': 'Airports',
             'American Restaurants': 'Fast Food Restaurants',
             'Asian Restaurants': 'Chinese Restaurants',
             'Asian Restaurants, Chinese Restaurants': 'Chinese Restaurants',
             'Auto Dealerships': 'Automotive Shops',
             'Auto Dealerships, Automotive Shops': 'Auto Dealerships',
             'Auto Garages, Auto Workshops, Automotive Shops': 'Automotive Shops',
             'Auto Garages, Automotive Shops': 'Automotive Shops',
             'Auto Workshops, Automotive Shops': 'Automotive Shops',
             'Automotive Shops, Gas Stations': 'Automotive Shops',
             'Bakeries, Cafés': 'Bakeries',
             'Banks, Buildings': 'Banks',
             'Banks, Offices': 'Banks',
             'Beaches, Surf Spots': 'Beaches',
             'Bed & Breakfasts': 'Hotels',
  

In [62]:
# special process
def get_lower(x):
    try:
        return x.lower()
    except:
        return x

for col in data.columns:
    if data[col].dtype == object and col != 'id':
        data[col] = data[col].apply(get_lower)

In [63]:
data.head(5)

,id,name,latitude,longitude,address,city,state,zip,country,url,phone,categories,point_of_interest,categories2
0,E_000001272c6c5d,café stad oudenaarde,50.859975,3.634196,abdijstraat,nederename,oost-vlaanderen,9700,be,NaN,NaN,bars,p_677e840bb6fc7e,bars
1,E_000002eae2a589,carioca manero,-22.907225,-43.178244,NaN,NaN,NaN,NaN,br,NaN,NaN,brazilian restaurants,p_d82910d8382a83,brazilian restaurants
2,E_000007f24ebc95,ร้านตัดผมการาเกด,13.780813,100.484900,NaN,NaN,NaN,NaN,th,NaN,NaN,salons / barbershops,p_b1066599e78477,salons / barbershops
3,E_000008a8ba4f48,turkcell,37.844510,27.844202,adnan menderes bulvarı,NaN,NaN,NaN,tr,NaN,NaN,mobile phone shops,p_b2ed86905a4cd3,mobile phone shops
4,E_00001d92066153,restaurante casa cofiño,43.338196,-4.326821,NaN,caviedes,cantabria,NaN,es,NaN,NaN,spanish restaurants,p_809a884d4407fb,spanish restaurants


In [64]:
## Data split
# ★★★★★★★★★★★★★★★
kf = GroupKFold(n_splits=2)
for i, (trn_idx, val_idx) in enumerate(kf.split(data, 
                                                data['point_of_interest'], 
                                                data['point_of_interest'])):
    if i == 0 :
      data.loc[val_idx, 'set'] = 0
    else :
      data.loc[val_idx, 'set'] = 1

print('Num of train data: %s' % len(data))
print(data['set'].value_counts())

valid_data = data[data['set'] == 0]
train_data = data[data['set'] == 1]

print('Train data: ')
analysis(train_data)
print('Valid data: ')
analysis(valid_data)

train_poi = train_data['point_of_interest'].unique().tolist()
valid_poi = valid_data['point_of_interest'].unique().tolist()

print(set(train_poi) & set(valid_poi))

train_ids = train_data['id'].unique().tolist()
valid_ids = valid_data['id'].unique().tolist()
      
print(set(train_ids) & set(valid_ids))
      
tv_ids_d = {}
tv_ids_d['train_ids'] = train_ids
tv_ids_d['valid_ids'] = valid_ids

np.save('tv_ids_d.npy', tv_ids_d)

del train_data, valid_data
gc.collect()

data = data.set_index('id')
data = data.loc[tv_ids_d['train_ids']]
data = data.reset_index()

Num of train data: 1138812
1.0    569406
0.0    569406
Name: set, dtype: int64
Train data: 
Num of data: 569406
Num of unique id: 569406
Num of unique poi: 369987
Mean num of unique poi: 1.5389892077289202
Valid data: 
Num of data: 569406
Num of unique id: 569406
Num of unique poi: 369985
Mean num of unique poi: 1.5389975269267673
set()
set()


In [65]:
len(data)

569406

In [66]:
%%time
## Train data generated by knn
id2index_d = dict(zip(data['id'].values, data.index))

tfidf_d = {}
for col in vec_columns:
    tfidf = TfidfVectorizer()
    tv_fit = tfidf.fit_transform(data[col].fillna('nan'))
    tfidf_d[col] = tv_fit

## Simple recall & knn recall
train_data_simple = recall_simple(data)
train_data, sel_dict = recall_knn(data, num_neighbors, num_neighbors2, num_neighbors3)

print('train data by knn: %s' % len(train_data))
train_data = train_data.merge(train_data_simple,
                             on = ['id', 'match_id'],
                             how = 'outer')
del train_data_simple
gc.collect()

# train_data, sel_dict = recall_knn(data, num_neighbors, num_neighbors2, num_neighbors3)

data = data.set_index('id')
ids = train_data['id'].tolist()
match_ids = train_data['match_id'].tolist()

poi = data.loc[ids]['point_of_interest'].values
match_poi = data.loc[match_ids]['point_of_interest'].values

train_data['label'] = np.array(poi == match_poi, dtype = np.int8)
del poi, match_poi, ids, match_ids
gc.collect()

print('Num of unique id: %s' % train_data['id'].nunique())
print('Num of train data: %s' % len(train_data))
print('Pos rate: %s' % train_data['label'].mean())
print("rate = ", len(train_data)/13928057)
print(train_data.sample(5))

  0%|          | 0/569406 [00:00<?, ?it/s]

Num of data: 1773340
Num of data per id: 4.748889995768862
Start knn grouped by country


  0%|          | 0/210 [00:00<?, ?it/s]

Start knn grouped by categories


  0%|          | 0/24447 [00:00<?, ?it/s]

Start knn
train data by knn: 18838749
Num of unique id: 569406
Num of train data: 20152823
Pos rate: 0.05297952549873534
rate =  1.4469227832712057
                        id          match_id  kneighbors  kneighbors_country  \
10552962  E_885c067ef05833  E_5a154fa4d0823a        18.0                 NaN   
17904999  E_85bd744cb824b6  E_4b20ce14d6b98e         NaN                 NaN   
16184584  E_6c4554fb6dc485  E_442c77f59d7753        28.0                 NaN   
16999688  E_dacfd79cb6b99e  E_ed54e86fd6e52d        29.0                 NaN   
4531354   E_f5368903908f28  E_66704f0e29d5bd         7.0                 6.0   

          kneighbors_categories  categories_index  label  
10552962                    NaN               NaN      0  
17904999                    2.0             236.0      0  
16184584                    NaN               NaN      0  
16999688                    NaN               NaN      0  
4531354                     NaN               NaN      0  
CPU times: user 2

In [67]:
sel_dict

{'accessories stores': 0,
 'advertising agencies': 1,
 'airport lounges': 2,
 'airport services': 3,
 'airport terminals': 4,
 'airports': 5,
 'american restaurants': 6,
 'american restaurants, burger joints': 7,
 'antique shops': 8,
 'aquariums': 9,
 'arcades': 10,
 'argentinian restaurants': 11,
 'art galleries': 12,
 'art museums': 13,
 'arts & crafts stores': 14,
 'arts & crafts stores, convenience stores, discount stores': 15,
 'asian restaurants': 16,
 'asian restaurants, chinese restaurants': 17,
 'asian restaurants, indonesian restaurants': 18,
 'asian restaurants, thai restaurants': 19,
 'assisted living': 20,
 'athletics & sports': 21,
 'atms': 22,
 'atms, banks': 23,
 'atms, banks, financial or legal services': 24,
 'auditoriums': 25,
 'auto dealerships': 26,
 'auto dealerships, automotive shops': 27,
 'auto garages': 28,
 'auto garages, auto workshops, automotive shops': 29,
 'auto garages, automotive shops': 30,
 'auto workshops': 31,
 'auto workshops, automotive shops': 3

In [68]:
# sum(train_data['kdist'].isna()), sum(train_data['kdist'].notna()),  len(train_data)

In [69]:
# 23341707
# 11952220

In [70]:
train_data.head()

,id,match_id,kneighbors,kneighbors_country,kneighbors_categories,categories_index,label
0,E_000001272c6c5d,E_000001272c6c5d,0.0,0.0,0.0,45.0,1
1,E_00009ab517afac,E_00009ab517afac,0.0,0.0,0.0,127.0,1
2,E_0000c362229d93,E_0000c362229d93,0.0,0.0,0.0,102.0,1
3,E_0000c566a81ea1,E_0000c566a81ea1,0.0,0.0,0.0,401.0,1
4,E_0000d9e584ed9f,E_0000d9e584ed9f,0.0,0.0,0.0,402.0,1


## 6/18 追加（反映チェック必要） kdistに距離を集約

In [71]:
# pos = train_data['kdist_country'].notna()
# train_data.loc[pos, 'kdist'] = train_data[pos]['kdist_country']

In [72]:
# pos = train_data['kdist_categories'].notna()
# train_data.loc[pos, 'kdist'] = train_data[pos]['kdist_categories']

In [73]:
# sum(train_data['kdist'].isna())

In [74]:
# sel = ['id', 'match_id', 'kdist', 'kneighbors',
#        'kneighbors_country', 'kneighbors_categories','categories_index',
#        'label']
# train_data = train_data[sel]

In [75]:
train_data.head()

,id,match_id,kneighbors,kneighbors_country,kneighbors_categories,categories_index,label
0,E_000001272c6c5d,E_000001272c6c5d,0.0,0.0,0.0,45.0,1
1,E_00009ab517afac,E_00009ab517afac,0.0,0.0,0.0,127.0,1
2,E_0000c362229d93,E_0000c362229d93,0.0,0.0,0.0,102.0,1
3,E_0000c566a81ea1,E_0000c566a81ea1,0.0,0.0,0.0,401.0,1
4,E_0000d9e584ed9f,E_0000d9e584ed9f,0.0,0.0,0.0,402.0,1


In [76]:
## Eval
data = data.reset_index()

id2poi = get_id2poi(data)
poi2ids = get_poi2ids(data)

eval_df = pd.DataFrame()
eval_df['id'] = data['id'].unique().tolist()
eval_df['match_id'] = eval_df['id']
print('Unique id: %s' % len(eval_df))

eval_df_ = train_data[train_data['label'] == 1][['id', 'match_id']]
eval_df = pd.concat([eval_df, eval_df_])

eval_df = eval_df.groupby('id')['match_id'].\
                        apply(list).reset_index()
eval_df['matches'] = eval_df['match_id'].apply(lambda x: ' '.join(set(x)))
print('Unique id: %s' % len(eval_df))

iou_score = get_score(eval_df)
print('IoU score: %s' % iou_score)

Unique id: 569406
Unique id: 569406
IoU score: 0.945588178796291


## IOU

In [77]:
print('IoU score: %s' % iou_score, num_neighbors, num_neighbors2, num_neighbors3,  len(train_data)/13928057)

IoU score: 0.945588178796291 10 6 30 1.4469227832712057


In [ ]:
  # IoU score: 0.9374236472867447 10 6 22 1.0282070930640217 NOW!!
# IoU score: 0.937478468485622 10 6 22 1.0298363942651871 0626 ほんと微妙にアップ（笑
# IoU score: 0.9401019072003999 10 6 22 1.1243562544294585 -- recall_simple追加　その他微修正
 # IoU score: 0.945588178796291 10 6 30 1.4469227832712057

In [ ]:
 # IoU score: 0.9210875384608563 20 20 1.0 --- original

In [ ]:
#IoU score: 0.936960291817873 1,39
#IoU score: 0.9344528470303082 5 35
#IoU score: 0.9375555341479278 3 40
#IoU score: 0.9375531925264253 2 40
#IoU score: 0.9375520217156739 1 40 1.6352776270229221
#IoU score: 0.9375555341479278 3 40 1.6353082127679404
#----------->  not radian #IoU score: 0.9375596682994936 3 40 1.6405100151442515
#IoU score: 0.9429661185642001 3 40
#IoU score: 0.9404926843925454 3 35 1.4721750492548962
#IoU score: 0.9352833892684943 10 20
#IoU score: 0.9434660236746832 5 30 1.5175960293671975
#IoU score: 0.9345203666132559 10 5 20 0.918320193548892
#IoU score: 0.9389112753713773 10 5 25 1.1197341452580212
#IoU score: 0.9347000717321834 10 5 20 0.920687932279427　
#IoU score: 0.9364244974115902 10 5 22 0.9987461998468271

In [ ]:
## Add features
count = 0
start_row = 0

data = data.set_index('id')
unique_id = train_data['id'].unique().tolist()
num_split_id = len(unique_id) // num_split
for k in range(1, num_split + 1):
    print('Current split: %s' % k)
    end_row = start_row + num_split_id
    if k < num_split:
        cur_id = unique_id[start_row : end_row]
        cur_data = train_data[train_data['id'].isin(cur_id)]
    else:
        cur_id = unique_id[start_row: ]
        cur_data = train_data[train_data['id'].isin(cur_id)]
    
    cur_data = add_features(cur_data)
    print(cur_data.shape)
    print(cur_data.sample(1))
    
    cur_data.to_csv('train_data%s.csv' % k, index = False)    
    print(cur_data.columns)
    start_row = end_row
    count += len(cur_data)
    
    del cur_data
    gc.collect()
    
print(count)

Current split: 1


  0%|          | 0/7 [00:00<?, ?it/s]

(2452937, 67)
                        id          match_id  kneighbors  kneighbors_country  \
17089855  E_024e6f1d00f75a  E_114568eb4dea60        20.0                 NaN   

          kneighbors_categories  categories_index  label   latdiff   londiff  \
17089855                    NaN               NaN      0 -0.028479  0.014031   

          manhattan  ...  categories2_sim  categories2_gesh  \
17089855   0.042511  ...              0.0          0.487805   

          categories2_leven  categories2_jaro  categories2_lcs  \
17089855               14.0          0.659916             10.0   

          categories2_fuz  categories2_len_diff  categories2_nleven  \
17089855             49.0              0.217391            0.608696   

          categories2_nlcsk  categories2_nlcs  
17089855           0.555556          0.434783  

[1 rows x 67 columns]
Index(['id', 'match_id', 'kneighbors', 'kneighbors_country',
       'kneighbors_categories', 'categories_index', 'label', 'latdiff',
       'l

  0%|          | 0/7 [00:00<?, ?it/s]

(2442137, 67)
                       id          match_id  kneighbors  kneighbors_country  \
6111960  E_27e32e474ddaf4  E_61f97a181ce7c1         7.0                 7.0   

         kneighbors_categories  categories_index  label   latdiff   londiff  \
6111960                    NaN               NaN      0 -0.000353 -0.000468   

         manhattan  ...  categories2_sim  categories2_gesh  categories2_leven  \
6111960   0.000821  ...              0.0          0.466667               15.0   

         categories2_jaro  categories2_lcs  categories2_fuz  \
6111960          0.838636              7.0             47.0   

         categories2_len_diff  categories2_nleven  categories2_nlcsk  \
6111960                  1.75            0.681818           0.318182   

         categories2_nlcs  
6111960             0.875  

[1 rows x 67 columns]
Index(['id', 'match_id', 'kneighbors', 'kneighbors_country',
       'kneighbors_categories', 'categories_index', 'label', 'latdiff',
       'londiff', 'ma

...

(2468215, 67)
                        id          match_id  kneighbors  kneighbors_country  \
23752812  E_c7a09ad3af7e47  E_76de241b01b310         NaN                 NaN   

          kneighbors_categories  categories_index  label    latdiff  \
23752812                    NaN               NaN      0 -18.418921   

            londiff  manhattan  ...  categories2_sim  categories2_gesh  \
23752812 -24.691015  43.109936  ...              1.0               1.0   

          categories2_leven  categories2_jaro  categories2_lcs  \
23752812                0.0               1.0             12.0   

          categories2_fuz  categories2_len_diff  categories2_nleven  \
23752812            100.0                   0.0                 0.0   

          categories2_nlcsk  categories2_nlcs  
23752812                1.0               1.0  

[1 rows x 67 columns]
Index(['id', 'match_id', 'kneighbors', 'kneighbors_country',
       'kneighbors_categories', 'categories_index', 'label', 'latdiff',
     

  0%|          | 0/7 [00:00<?, ?it/s]

(2431456, 67)
                       id          match_id  kneighbors  kneighbors_country  \
3291591  E_da6f3f16b2157a  E_8497b07973b5dd         3.0                 4.0   

         kneighbors_categories  categories_index  label   latdiff   londiff  \
3291591                    NaN               NaN      0  0.008235 -0.003472   

         manhattan  ...  categories2_sim  categories2_gesh  categories2_leven  \
3291591   0.011707  ...              0.0          0.230769               35.0   

         categories2_jaro  categories2_lcs  categories2_fuz  \
3291591          0.515798              7.0             27.0   

         categories2_len_diff  categories2_nleven  categories2_nlcsk  \
3291591              2.727273            0.853659           0.170732   

         categories2_nlcs  
3291591          0.636364  

[1 rows x 67 columns]
Index(['id', 'match_id', 'kneighbors', 'kneighbors_country',
       'kneighbors_categories', 'categories_index', 'label', 'latdiff',
       'londiff', 'ma

  0%|          | 0/7 [00:00<?, ?it/s]

(2456829, 67)
                        id          match_id  kneighbors  kneighbors_country  \
17074324  E_fda5364b6c68fc  E_c5c321a9dced1b        19.0                 NaN   

          kneighbors_categories  categories_index  label   latdiff   londiff  \
17074324                    NaN               NaN      0 -0.010185 -0.011877   

          manhattan  ...  categories2_sim  categories2_gesh  \
17074324   0.022061  ...              0.0          0.344828   

          categories2_leven  categories2_jaro  categories2_lcs  \
17074324               16.0          0.396825              5.0   

          categories2_fuz  categories2_len_diff  categories2_nleven  \
17074324             34.0              0.619048            0.761905   

          categories2_nlcsk  categories2_nlcs  
17074324              0.625          0.238095  

[1 rows x 67 columns]
Index(['id', 'match_id', 'kneighbors', 'kneighbors_country',
       'kneighbors_categories', 'categories_index', 'label', 'latdiff',
       'l

In [ ]:
!cp * /content/drive/MyDrive/datas/foursquare/traindata0628